# Seq2seq model - pitch-based
- [_Joplin_](http://kern.humdrum.org/search?s=t&keyword=Joplin) dataset is used as the training data for this model.
- A song is parsed into a sequence (containing one or multiple measures) of chords. 
    - Chords are represented in sequence of pitches, separated by '/', thus this model is called pitch-based; i.e., chords are represented as a sequence of one hot vectors that representing different pitches. 
    - In [chord-based model](Seq2seq model - Joplin.ipynb), each chord is represented as a single multi-hot vector.

In [1]:
import os
import util

## Part 1 - chord sequence to chord sequence
### Load data
- _Joplin_ dataset is used. 
- The data is parsed from kern score using <code>rchoraledata</code> tool in [_Humdrum Extras_](http://wiki.ccarh.org/wiki/Humdrum_Extras). See the [embedding notebook](Chord_Embeddings - Joplin.ipynb) for details.
- Filtering data
    - Only songs in 2/4 time signature are selected since most of the songs in this dataset are in 2/4.

In [2]:
music_type = 'joplin'
dataset = 'datasets/' + music_type

In [3]:
files = [f for f in os.listdir(dataset+'/dat/') if f.endswith('.dat')]
files = files.sort()

In [4]:
files = [f for f in os.listdir(dataset+'/krn/') if f.endswith('.krn')]
filtered_files = [] # only use song with time sig 2/4

for file in files[:]:
    basename = file[:-4]
    #! transpose -k c $dataset/krn/$basename\.krn | rchoraledata > $dataset/dat/$basename\.dat
    ret = ! beat -Ca $dataset/krn/$basename\.krn | beat -Ua  | extractx -s '$1-$'  | ridx -H | sortcount -p
    if ret[0] == '100\t2\t4': filtered_files.append(file)

In [5]:
filtered_files.sort()
filtered_files = [f[:-4] + '.dat' for f in filtered_files]

### Build dictionaries for pitch and time information
- Indexing different pitches and time durations.

In [6]:
# c_data: [song, measure, chord, pitch]
# t_data: [song, measure, dur]
t_data, c_data, t_val2idx, t_idx2val, c_val2idx, c_idx2val = (util.parse_dat_dir(dataset, filtered_files))

#### Add start, delimeter, and end tokens to the dictionaries
- <code>S</code> -  the start of a sequence
- <code>END</code> - the end of a sequence
- <code>/</code> - a delimeter that separates different chords

In [7]:
c_val2idx['<S>'] = len(c_val2idx)
c_idx2val[c_val2idx['<S>']] = '<S>'
c_val2idx['/'] = len(c_val2idx)
c_idx2val[c_val2idx['/']] = '/'
c_val2idx['<END>'] = len(c_val2idx)
c_idx2val[c_val2idx['<END>']] = '<END>'

t_val2idx['<S>'] = len(t_val2idx)
t_idx2val[t_val2idx['<S>']] = '<S>'
t_val2idx['<END>'] = len(t_val2idx)
t_idx2val[t_val2idx['<END>']] = '<END>'

In [8]:
meas_per_seq = 2
encoder_input_data, decoder_input_data, decoder_target_data = util.gen_data_rec_pitch_based(c_data, c_val2idx, meas_per_seq=meas_per_seq) #[song, measure, chord]

### Generate input and target data that fits the model requirements
- The data is in the form of a list of 3D arrays.
    - Each entry in the list represents a song.
    - The 3 dimensions are for <code>[num_seq, max_seq_len, num_features]</code>.
    - Padded with zeros if <code>max_seq_len</code> is not met.

In [9]:
encoder_input_data = util.get_data_fitted(encoder_input_data)
decoder_input_data = util.get_data_fitted(decoder_input_data)
decoder_target_data = util.get_data_fitted(decoder_target_data)

### Build the seq2seq model - predict next chords sequence from current chords sequence
- Referenced from a [Keras seq2seq tutorial](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html).
- Have a look at [Keras](https://keras.io), which is a easy-to-use neural network API.

In [10]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
iterations = 40  # Number of epochs to train for.
latent_dim = 256 

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
num_encoder_tokens = len(c_val2idx)
num_decoder_tokens = num_encoder_tokens
max_encoder_seq_length = None # different max-seq-len for different songs
max_decoder_seq_length = None

In [12]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [13]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [14]:
#model.reset_states()
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [15]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 142)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 142)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 408576      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  408576      input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

### Start training

In [17]:
batch_size = 64  # Batch size for training.
iterations = 55  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

for it in range(iterations):
    print('='*50)
    print('Iteration {}/{}'.format(it + 1, iterations))
    for i in range(len(c_data)):
        model.fit([encoder_input_data[i], decoder_input_data[i]], decoder_target_data[i],
              batch_size=None,
              epochs=1,
              validation_split=0.01)

Iteration 1/55
Train on 97 samples, validate on 1 samples
Epoch 1/1
97/97 [==============================] - 3s 30ms/step - loss: 3.2603 - val_loss: 1.6568
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 20ms/step - loss: 2.9637 - val_loss: 1.7581
Train on 100 samples, validate on 2 samples
Epoch 1/1
100/100 [==============================] - 2s 17ms/step - loss: 2.8041 - val_loss: 2.9772
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 16ms/step - loss: 2.9865 - val_loss: 1.9160
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 2s 21ms/step - loss: 2.4011 - val_loss: 1.5660
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 16ms/step - loss: 2.9366 - val_loss: 2.4156
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 2s 17ms/step - loss: 2.8439 - val_loss: 1.

86/86 [==============================] - 1s 17ms/step - loss: 2.4313 - val_loss: 1.3574
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 16ms/step - loss: 2.6665 - val_loss: 1.8957
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 17ms/step - loss: 2.4235 - val_loss: 1.7105
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 22ms/step - loss: 2.6458 - val_loss: 1.6351
Train on 29 samples, validate on 1 samples
Epoch 1/1
29/29 [==============================] - 1s 30ms/step - loss: 1.8216 - val_loss: 0.9109
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 2s 20ms/step - loss: 2.6383 - val_loss: 1.5039
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 24ms/step - loss: 2.0415 - val_loss: 1.0970
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [======

89/89 [==============================] - 2s 21ms/step - loss: 1.8575 - val_loss: 1.1664
Train on 99 samples, validate on 2 samples
Epoch 1/1
99/99 [==============================] - 2s 17ms/step - loss: 1.9163 - val_loss: 1.5440
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 11ms/step - loss: 1.8939 - val_loss: 1.5492
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 19ms/step - loss: 1.7813 - val_loss: 1.1743
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 21ms/step - loss: 1.8716 - val_loss: 0.9952
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 1s 16ms/step - loss: 2.5068 - val_loss: 1.7498
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 2s 19ms/step - loss: 2.0397 - val_loss: 1.5331
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [======

Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 15ms/step - loss: 1.5515 - val_loss: 0.8271
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 15ms/step - loss: 1.4456 - val_loss: 0.8309
Train on 83 samples, validate on 1 samples
Epoch 1/1
83/83 [==============================] - 1s 17ms/step - loss: 1.6424 - val_loss: 0.7044
Train on 90 samples, validate on 1 samples
Epoch 1/1
90/90 [==============================] - 2s 17ms/step - loss: 1.7308 - val_loss: 1.3348
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 15ms/step - loss: 1.5618 - val_loss: 1.3827
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 1s 21ms/step - loss: 1.9972 - val_loss: 1.0744
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 18ms/step - loss: 1.4747 - val_loss: 0.8482
Train on 91 s

81/81 [==============================] - 1s 17ms/step - loss: 1.5436 - val_loss: 0.8704
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 1s 16ms/step - loss: 1.3235 - val_loss: 1.1150
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 2s 24ms/step - loss: 1.5444 - val_loss: 0.9270
Train on 106 samples, validate on 2 samples
Epoch 1/1
106/106 [==============================] - 3s 29ms/step - loss: 1.5359 - val_loss: 1.1713
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 2s 25ms/step - loss: 1.5307 - val_loss: 0.8926
Train on 105 samples, validate on 2 samples
Epoch 1/1
105/105 [==============================] - 2s 15ms/step - loss: 1.5866 - val_loss: 1.4685
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 17ms/step - loss: 1.2914 - val_loss: 1.0778
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [

...

Train on 83 samples, validate on 1 samples
Epoch 1/1
83/83 [==============================] - 2s 25ms/step - loss: 0.2629 - val_loss: 0.2819
Train on 90 samples, validate on 1 samples
Epoch 1/1
90/90 [==============================] - 2s 19ms/step - loss: 0.1982 - val_loss: 0.9414
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 18ms/step - loss: 0.2184 - val_loss: 1.5698
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 2s 25ms/step - loss: 0.3134 - val_loss: 0.6176
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 19ms/step - loss: 0.2966 - val_loss: 0.3206
Train on 91 samples, validate on 1 samples
Epoch 1/1
91/91 [==============================] - 1s 16ms/step - loss: 0.3281 - val_loss: 0.4255
Iteration 49/55
Train on 97 samples, validate on 1 samples
Epoch 1/1
97/97 [==============================] - 2s 17ms/step - loss: 0.2770 - val_loss: 0.27

81/81 [==============================] - 1s 15ms/step - loss: 0.2489 - val_loss: 0.6249
Train on 106 samples, validate on 2 samples
Epoch 1/1
106/106 [==============================] - 2s 15ms/step - loss: 0.2172 - val_loss: 1.3107
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 1s 18ms/step - loss: 0.2678 - val_loss: 0.9233
Train on 105 samples, validate on 2 samples
Epoch 1/1
105/105 [==============================] - 1s 13ms/step - loss: 0.2732 - val_loss: 1.5111
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 15ms/step - loss: 0.2371 - val_loss: 0.2931
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 16ms/step - loss: 0.3333 - val_loss: 0.4719
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 15ms/step - loss: 0.2914 - val_loss: 0.7038
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [

97/97 [==============================] - 2s 17ms/step - loss: 0.2338 - val_loss: 0.2615
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 17ms/step - loss: 0.3415 - val_loss: 0.7800
Train on 100 samples, validate on 2 samples
Epoch 1/1
100/100 [==============================] - 1s 15ms/step - loss: 0.2260 - val_loss: 0.5931
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 14ms/step - loss: 0.2043 - val_loss: 0.4738
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 1s 16ms/step - loss: 0.2575 - val_loss: 0.7603
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 14ms/step - loss: 0.1876 - val_loss: 0.7933
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 1s 15ms/step - loss: 0.2053 - val_loss: 0.6007
Train on 99 samples, validate on 2 samples
Epoch 1/1
99/99 [===

Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 14ms/step - loss: 0.2298 - val_loss: 0.9012
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 14ms/step - loss: 0.2342 - val_loss: 1.2069
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 17ms/step - loss: 0.2782 - val_loss: 0.8631
Train on 29 samples, validate on 1 samples
Epoch 1/1
29/29 [==============================] - 1s 25ms/step - loss: 0.3931 - val_loss: 0.4071
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 1s 16ms/step - loss: 0.2360 - val_loss: 1.0023
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 20ms/step - loss: 0.2950 - val_loss: 0.7669
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 13ms/step - loss: 0.2001 - val_loss: 0.3943
Train on 84 s

99/99 [==============================] - 2s 16ms/step - loss: 0.1751 - val_loss: 0.5485
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 12ms/step - loss: 0.1464 - val_loss: 0.7925
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 18ms/step - loss: 0.1418 - val_loss: 0.1482
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 2s 19ms/step - loss: 0.2672 - val_loss: 0.7113
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 1s 16ms/step - loss: 0.2602 - val_loss: 0.9051
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 2s 18ms/step - loss: 0.2375 - val_loss: 0.5973
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 1s 15ms/step - loss: 0.1958 - val_loss: 0.4839
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [======

In [18]:
save_name = 'models/no_emb_joplin_' + str(meas_per_seq) + '.h5'
model.save(save_name)

/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


### Sampling the notes
- Different sampling methods are used.
    - $\arg\max$: selecting the pitch with highest probability.
        - Deterministic.
        - Potential problem: always choose the most frequent values, the output tends to be repetitive.
    - Temperature sampling with different $\tau$s.
        - Randomly choose based on temperature sampling; stochastic.
        - Higher $\tau$s adds variety to the music generated.
        - Potential problem: large $\tau$ increases the chance of inharmony.
- Sampling methods can be seen at [sampling.py](sampling.py).

In [19]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = c_idx2val
reverse_target_char_index = c_idx2val
input_token_index = c_val2idx
target_token_index = c_val2idx

In [20]:
import sampling

In [21]:
def decode_sequence(input_seq, samp_fun=sampling.argmax, max_decoder_seq_length=300):
     
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['<S>']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    target_ret = []
    target_ret.append(target_seq[0, 0, :])
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = samp_fun(output_tokens[0, -1, :]) # sampling using argmax
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += str(sampled_char) + ' '

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '<END>' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        target_ret.append(target_seq[0, 0, :])

        # Update states
        states_value = [h, c]

    return decoded_sentence, np.array([np.array(target_ret)])

In [22]:
def print_seq(input_seq, display_name=True):
    
    naming_func = lambda x: x
    if display_name:
        naming_func = util.get_note_name_by_idx

    sent = ''
    
    for measure in input_seq[0,:,:] :
        indices = np.where(measure > 0)[0]
        if len(indices != 0):
            idx = indices[0]
            sampled_char = reverse_target_char_index[idx]
            if sampled_char == '<S>' or sampled_char == '<END>' or sampled_char == '/': sent += sampled_char
            elif sampled_char >= 0:
                sent += str(naming_func(sampled_char))
            else:
                sent += '-' + str(naming_func(np.abs(sampled_char)))
            sent += ' '
        else: break
    
    return sent

In [27]:
target_token_index = c_val2idx

chords_gen_argmax = []

for seq_index in range(len(encoder_input_data)): # which song
    
    curr_gen = []
    
    filename = 'output/joplin/chords/argmax/' + str(meas_per_seq) + '_' + str(seq_index).zfill(2) + '.txt'
    f = open(filename, 'w')

    input_seq = np.array([encoder_input_data[seq_index][0]]) # input is the first measure
    f.write(print_seq(input_seq, display_name=False) + '\n')
    curr_gen.append(input_seq)
    
    decoded_sentence, target_ret = decode_sequence(input_seq)
    f.write(print_seq(target_ret, display_name=False) + '\n')
    curr_gen.append(target_ret)
    
    for i in range(64):
        input_seq = np.copy(target_ret)
        decoded_sentence, target_ret = decode_sequence(input_seq)
        f.write(print_seq(target_ret, display_name=False) + '\n')
        curr_gen.append(target_ret)

    f.close()
    chords_gen_argmax.append(curr_gen)

In [28]:
chords_gen_tau1 = []

for seq_index in range(len(encoder_input_data)): # which song
    
    curr_gen = []
    
    filename = 'output/joplin/chords/tau1.0/' + str(meas_per_seq) + '_' + str(seq_index).zfill(2) + '.txt'
    f = open(filename, 'w')

    input_seq = np.array([encoder_input_data[seq_index][0]]) # input is the first measure
    f.write(print_seq(input_seq, display_name=False) + '\n')
    curr_gen.append(input_seq)
    
    decoded_sentence, target_ret = decode_sequence(input_seq, samp_fun=sampling.temperature(tau=1.0))
    f.write(print_seq(target_ret, display_name=False) + '\n')
    curr_gen.append(target_ret)
    
    for i in range(64):
        input_seq = np.copy(target_ret)
        decoded_sentence, target_ret = decode_sequence(input_seq, samp_fun=sampling.temperature(tau=1.0))
        f.write(print_seq(target_ret, display_name=False) + '\n')
        curr_gen.append(target_ret)

    f.close()
    chords_gen_tau1.append(curr_gen)

In [29]:
chords_gen_tau2 = []

for seq_index in range(len(encoder_input_data)): # which song
    
    curr_gen = []
    
    filename = 'output/joplin/chords/tau0.9/' + str(meas_per_seq) + '_' + str(seq_index).zfill(2) + '.txt'
    f = open(filename, 'w')

    input_seq = np.array([encoder_input_data[seq_index][0]]) # input is the first measure
    f.write(print_seq(input_seq, display_name=False) + '\n')
    curr_gen.append(input_seq)
    
    decoded_sentence, target_ret = decode_sequence(input_seq, samp_fun=sampling.temperature(tau=0.9))
    f.write(print_seq(target_ret, display_name=False) + '\n')
    curr_gen.append(target_ret)
    
    for i in range(64):
        input_seq = np.copy(target_ret)
        decoded_sentence, target_ret = decode_sequence(input_seq, samp_fun=sampling.temperature(tau=0.9))
        f.write(print_seq(target_ret, display_name=False) + '\n')
        curr_gen.append(target_ret)

    f.close()
    chords_gen_tau2.append(curr_gen)

In [30]:
chords_gen_tau3 = []

for seq_index in range(len(encoder_input_data)): # which song
    
    curr_gen = []
    
    filename = 'output/joplin/chords/tau0.8/' + str(meas_per_seq) + '_' + str(seq_index).zfill(2) + '.txt'
    f = open(filename, 'w')

    input_seq = np.array([encoder_input_data[seq_index][0]]) # input is the first measure
    f.write(print_seq(input_seq, display_name=False) + '\n')
    curr_gen.append(input_seq)
    
    decoded_sentence, target_ret = decode_sequence(input_seq, samp_fun=sampling.temperature(tau=0.8))
    f.write(print_seq(target_ret, display_name=False) + '\n')
    curr_gen.append(target_ret)
    
    for i in range(64):
        input_seq = np.copy(target_ret)
        decoded_sentence, target_ret = decode_sequence(input_seq, samp_fun=sampling.temperature(tau=0.8))
        f.write(print_seq(target_ret, display_name=False) + '\n')
        curr_gen.append(target_ret)

    f.close()
    chords_gen_tau3.append(curr_gen)

## Part 2 - chord sequence to time sequence
### Load data
- _Joplin_ dataset is used. 
- The data is parsed from kern score using <code>rchoraledata</code> tool in [_Humdrum Extras_](http://wiki.ccarh.org/wiki/Humdrum_Extras). See the [embedding notebook](Chord_Embeddings - Joplin.ipynb) for details.
- Filtering data
    - Only songs in 2/4 time signature are selected since most of the songs in this dataset are in 2/4.

In [31]:
encoder_input_data, decoder_input_data, decoder_target_data = util.gen_data_rec_pitch_based_times(c_data, c_val2idx, t_data, t_val2idx, meas_per_seq=meas_per_seq) #[song, measure, chord]

In [32]:
encoder_input_data = util.get_data_fitted(encoder_input_data)
decoder_input_data = util.get_data_fitted(decoder_input_data)
decoder_target_data = util.get_data_fitted(decoder_target_data)

In [33]:
num_encoder_tokens = len(c_val2idx)
num_decoder_tokens = len(t_val2idx)
max_encoder_seq_length = None # different max-seq-len for different songs
max_decoder_seq_length = None

In [34]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [35]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [36]:
model.reset_states()
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [37]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [38]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, 142)    0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None, 12)     0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 408576      input_5[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 256),  275456      input_6[0][0]                    
                                                                 lstm_3[0][1]                     
          

In [39]:
batch_size = 64  # Batch size for training.
iterations = 50  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

for it in range(iterations):
    print('='*50)
    print('Iteration {}/{}'.format(it + 1, iterations))
    for i in range(len(c_data)):
        model.fit([encoder_input_data[i], decoder_input_data[i]], decoder_target_data[i],
              batch_size=None,
              epochs=1,
              validation_split=0.01)

Iteration 1/50
Train on 97 samples, validate on 1 samples
Epoch 1/1
97/97 [==============================] - 2s 23ms/step - loss: 1.7217 - val_loss: 0.6637
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 9ms/step - loss: 0.7458 - val_loss: 0.8910
Train on 100 samples, validate on 2 samples
Epoch 1/1
100/100 [==============================] - 1s 8ms/step - loss: 0.7239 - val_loss: 0.8454
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 7ms/step - loss: 0.7540 - val_loss: 0.4812
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 1s 11ms/step - loss: 0.6071 - val_loss: 0.5885
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 10ms/step - loss: 0.4775 - val_loss: 1.0872
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 1s 10ms/step - loss: 0.6359 - val_loss: 0.765

86/86 [==============================] - 1s 9ms/step - loss: 0.5757 - val_loss: 0.4400
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 9ms/step - loss: 0.4656 - val_loss: 0.6326
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 8ms/step - loss: 0.4806 - val_loss: 0.4928
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 12ms/step - loss: 0.3952 - val_loss: 1.0387
Train on 29 samples, validate on 1 samples
Epoch 1/1
29/29 [==============================] - 0s 15ms/step - loss: 0.4915 - val_loss: 0.7419
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 1s 10ms/step - loss: 0.4587 - val_loss: 0.7218
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 12ms/step - loss: 2.4020 - val_loss: 0.2793
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [=========

89/89 [==============================] - 1s 10ms/step - loss: 0.5065 - val_loss: 0.4678
Train on 99 samples, validate on 2 samples
Epoch 1/1
99/99 [==============================] - 1s 11ms/step - loss: 0.3424 - val_loss: 0.7272
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 7ms/step - loss: 0.8414 - val_loss: 0.4026
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 11ms/step - loss: 0.4356 - val_loss: 0.3454
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 10ms/step - loss: 0.4855 - val_loss: 0.5549
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 1s 9ms/step - loss: 0.4416 - val_loss: 0.6754
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 9ms/step - loss: 0.5065 - val_loss: 0.4084
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [=========

84/84 [==============================] - 1s 6ms/step - loss: 0.7194 - val_loss: 0.3464
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 7ms/step - loss: 0.5436 - val_loss: 0.3001
Train on 83 samples, validate on 1 samples
Epoch 1/1
83/83 [==============================] - 1s 8ms/step - loss: 0.3947 - val_loss: 0.2913
Train on 90 samples, validate on 1 samples
Epoch 1/1
90/90 [==============================] - 1s 7ms/step - loss: 0.3081 - val_loss: 0.3216
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 6ms/step - loss: 0.4495 - val_loss: 0.5574
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 1s 10ms/step - loss: 0.3931 - val_loss: 0.6042
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 8ms/step - loss: 0.3967 - val_loss: 0.4213
Train on 91 samples, validate on 1 samples
Epoch 1/1
91/91 [============

81/81 [==============================] - 1s 8ms/step - loss: 0.3239 - val_loss: 0.6170
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 1s 7ms/step - loss: 0.5086 - val_loss: 0.3984
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 1s 7ms/step - loss: 0.3452 - val_loss: 0.4210
Train on 106 samples, validate on 2 samples
Epoch 1/1
106/106 [==============================] - 1s 8ms/step - loss: 0.4358 - val_loss: 0.4136
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 1s 9ms/step - loss: 0.3685 - val_loss: 0.3650
Train on 105 samples, validate on 2 samples
Epoch 1/1
105/105 [==============================] - 1s 7ms/step - loss: 0.5151 - val_loss: 0.2870
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 8ms/step - loss: 0.3986 - val_loss: 0.4422
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [=======

...

100/100 [==============================] - 1s 8ms/step - loss: 0.2530 - val_loss: 0.3108
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 7ms/step - loss: 0.1914 - val_loss: 0.3756
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 1s 8ms/step - loss: 0.2790 - val_loss: 0.6951
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 7ms/step - loss: 0.1489 - val_loss: 0.2859
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 1s 9ms/step - loss: 0.2532 - val_loss: 0.3558
Train on 99 samples, validate on 2 samples
Epoch 1/1
99/99 [==============================] - 1s 9ms/step - loss: 0.1856 - val_loss: 0.2848
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 6ms/step - loss: 0.2654 - val_loss: 0.2721
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [===========

85/85 [==============================] - 1s 12ms/step - loss: 0.1590 - val_loss: 0.6784
Train on 29 samples, validate on 1 samples
Epoch 1/1
29/29 [==============================] - 1s 24ms/step - loss: 0.2429 - val_loss: 0.5154
Train on 87 samples, validate on 1 samples
Epoch 1/1
87/87 [==============================] - 1s 13ms/step - loss: 0.2454 - val_loss: 0.5761
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 15ms/step - loss: 0.2312 - val_loss: 0.2154
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 8ms/step - loss: 0.1706 - val_loss: 0.3166
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 10ms/step - loss: 0.2792 - val_loss: 0.1177
Train on 83 samples, validate on 1 samples
Epoch 1/1
83/83 [==============================] - 1s 11ms/step - loss: 0.2027 - val_loss: 0.2071
Train on 90 samples, validate on 1 samples
Epoch 1/1
90/90 [=======

73/73 [==============================] - 1s 8ms/step - loss: 0.2070 - val_loss: 0.1802
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 10ms/step - loss: 0.2308 - val_loss: 0.2559
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 1s 8ms/step - loss: 0.1863 - val_loss: 0.6244
Train on 84 samples, validate on 1 samples
Epoch 1/1
84/84 [==============================] - 1s 11ms/step - loss: 0.2157 - val_loss: 0.2122
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 1s 10ms/step - loss: 0.1613 - val_loss: 0.3831
Train on 89 samples, validate on 1 samples
Epoch 1/1
89/89 [==============================] - 1s 7ms/step - loss: 0.2829 - val_loss: 0.1381
Train on 81 samples, validate on 1 samples
Epoch 1/1
81/81 [==============================] - 1s 7ms/step - loss: 0.1750 - val_loss: 0.1609
Train on 106 samples, validate on 2 samples
Epoch 1/1
106/106 [=======

90/90 [==============================] - 1s 8ms/step - loss: 0.1013 - val_loss: 0.1399
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 7ms/step - loss: 0.1720 - val_loss: 0.2849
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 1s 9ms/step - loss: 0.1955 - val_loss: 0.6268
Train on 85 samples, validate on 1 samples
Epoch 1/1
85/85 [==============================] - 1s 11ms/step - loss: 0.2095 - val_loss: 0.2494
Train on 91 samples, validate on 1 samples
Epoch 1/1
91/91 [==============================] - 1s 10ms/step - loss: 0.1968 - val_loss: 0.4227
Iteration 49/50
Train on 97 samples, validate on 1 samples
Epoch 1/1
97/97 [==============================] - 1s 9ms/step - loss: 0.2129 - val_loss: 0.6014
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 8ms/step - loss: 0.2617 - val_loss: 0.5228
Train on 100 samples, validate on 2 samples
Epoch 1/1
1

106/106 [==============================] - 1s 8ms/step - loss: 0.1881 - val_loss: 0.4536
Train on 69 samples, validate on 1 samples
Epoch 1/1
69/69 [==============================] - 1s 9ms/step - loss: 0.1835 - val_loss: 0.1267
Train on 105 samples, validate on 2 samples
Epoch 1/1
105/105 [==============================] - 1s 7ms/step - loss: 0.2859 - val_loss: 0.0808
Train on 73 samples, validate on 1 samples
Epoch 1/1
73/73 [==============================] - 1s 7ms/step - loss: 0.1804 - val_loss: 0.1621
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 8ms/step - loss: 0.1778 - val_loss: 0.2644
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 7ms/step - loss: 0.1983 - val_loss: 0.2754
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [==============================] - 1s 7ms/step - loss: 0.1808 - val_loss: 0.1152
Train on 86 samples, validate on 1 samples
Epoch 1/1
86/86 [========

In [40]:
save_name = 'models/no_emb_joplin_times' + str(meas_per_seq) + '.h5'
model.save(save_name)

/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:2368: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [41]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = c_idx2val
reverse_target_char_index = t_idx2val
input_token_index = c_val2idx
target_token_index = t_val2idx

In [43]:
# generate from argmax chords
for i, song in enumerate(chords_gen_argmax): # which song
    
    filename = 'output/joplin/times/argmax/' + str(meas_per_seq) + '_' + str(i).zfill(2) + '.txt'
    f = open(filename, 'w')

    for seq in song:
        input_seq = np.array(seq) # input is the first measure
        decoded_sentence, target_ret = decode_sequence(input_seq)
        f.write(print_seq(target_ret, display_name=False) + '\n')

    f.close()

In [51]:
reverse_target_char_index = t_idx2val
# generate from tau1.0 chords
for i, song in enumerate(chords_gen_tau1): # which song
    
    filename = 'output/joplin/times/tau1.0/' + str(meas_per_seq) + '_' + str(i).zfill(2) + '.txt'
    f = open(filename, 'w')

    for seq in song:
        input_seq = np.array(seq) # input is the first measure
        decoded_sentence, target_ret = decode_sequence(input_seq)
        f.write(print_seq(target_ret, display_name=False) + '\n')

    f.close()

In [52]:
# generate from tau0.9 chords
for i, song in enumerate(chords_gen_tau2): # which song
    
    filename = 'output/joplin/times/tau0.9/' + str(meas_per_seq) + '_' + str(i).zfill(2) + '.txt'
    f = open(filename, 'w')

    for seq in song:
        input_seq = np.array(seq) # input is the first measure
        decoded_sentence, target_ret = decode_sequence(input_seq)
        f.write(print_seq(target_ret, display_name=False) + '\n')

    f.close()

In [53]:
# generate from tau0.8 chords
for i, song in enumerate(chords_gen_tau3): # which song
    
    filename = 'output/joplin/times/tau0.8/' + str(meas_per_seq) + '_' + str(i).zfill(2) + '.txt'
    f = open(filename, 'w')

    for seq in song:
        input_seq = np.array(seq) # input is the first measure
        decoded_sentence, target_ret = decode_sequence(input_seq)
        f.write(print_seq(target_ret, display_name=False) + '\n')

    f.close()